In [ ]:
from logic.camera import Camera


cams = Camera.list()
print("\n".join([str(cam) for cam in cams]))

In [ ]:
import cv2

from logic.state import DoubleImage

def make_image() -> DoubleImage | None:
    cam1 = cams[1].connect()
    cam2 = cams[2].connect()

    running = True

    cap_image: DoubleImage | None = None
    capture = False

    while running:
        if cap_image is None:
            has_frame1, frame1 = cam1.capture.read()
            has_frame2, frame2 = cam2.capture.read()
        else:
            frame1 = cap_image.image1
            frame2 = cap_image.image2
            has_frame1 = has_frame2 = True

        if has_frame1:        
            cv2.imshow("Cam 1", frame1)

        if has_frame2:        
            cv2.imshow("Cam 2", frame2)    

        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            running = False
        elif key == ord('c'):
            capture = True
        elif key == ord('d'):
            cap_image = None

        if has_frame1 and has_frame2 and capture:
            cap_image = DoubleImage(frame1, frame2)
            capture = False

    cv2.destroyWindow("Cam 1")
    cv2.destroyWindow("Cam 2")

    return cap_image

captured_calibration_image = make_image()
captured_hand_image = make_image()

if captured_calibration_image is not None:
    print("Images Captured") 
else:
    print("No Image")

In [ ]:
from typing import Sequence, cast
import numpy as np
from cv2.typing import MatLike

patternSize = (6, 9)
squareWidth = 24.3

height1, width1 = captured_calibration_image.image1.shape[:2]
height2, width2 = captured_calibration_image.image2.shape[:2]
imageSize = np.array((min(width1, width2), min(height1, height2)), dtype=np.int32)

scaledImage1 = cv2.resize(captured_calibration_image.image1, imageSize)
scaledImage2 = cv2.resize(captured_calibration_image.image2, imageSize)

objectPoints = np.array([[row * squareWidth, column * squareWidth, 0] for row in range(patternSize[0]) for column in range(patternSize[1])], dtype=np.float32)

cameraMatrix1 = np.array([
    [imageSize[0], 0, imageSize[0] / 2],
    [0, imageSize[1], imageSize[1] / 2],
    [0, 0, 1]
], dtype=np.float64)
distCoeffs1 = np.zeros(5, dtype=np.float64)
cameraMatrix2 = cameraMatrix1.copy()
distCoeffs2 = np.zeros(5, dtype=np.float64)

retval1, corners1 = cast(
    typ=tuple[bool, MatLike],
    val=cv2.findChessboardCorners(scaledImage1, patternSize)
)

retval2, corners2 = cast(
    typ=tuple[bool, MatLike],
    val=cv2.findChessboardCorners(scaledImage2, patternSize)
)

print("got points", retval1, retval2)

retval, cameraMatrix1, distCoeffs1, rvecs1, tvecs1 = cast(
    typ=tuple[float, MatLike, MatLike, Sequence[MatLike], Sequence[MatLike]],
    val=cv2.calibrateCamera([objectPoints], [corners1], imageSize, cameraMatrix1, distCoeffs1)
)

print("calibrated 1", retval)

retval, cameraMatrix2, distCoeffs2, rvecs2, tvecs2 = cast(
    typ=tuple[float, MatLike, MatLike, Sequence[MatLike], Sequence[MatLike]],
    val=cv2.calibrateCamera([objectPoints], [corners2], imageSize, cameraMatrix2, distCoeffs2)
)

print("calibrated 2", retval)

retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F = cast(
    typ=tuple[float, MatLike, MatLike, MatLike, MatLike, MatLike, MatLike, MatLike, MatLike],
    val=cv2.stereoCalibrate([objectPoints], [corners1], [corners2], cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize)
)

print("calibrated Both", retval)


In [ ]:
print("HEY")
output = captured_hand_image.image1.copy()
gray = cv2.cvtColor(captured_hand_image.image1, cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 16, 20, minRadius=3, maxRadius=20)

if circles is None:
    raise Exception("No Circles found")

circles = np.round(circles[0, :]).astype("int")
for (x, y, r) in circles:
    cv2.circle(output, (x, y), r, (0, 255, 0), 4)
    cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)

cv2.imshow("HEY", output)
cv2.waitKey(1)


- Start with params
- Make Calibration Picture
- Make Hand Pictures (Points are automatically recognized and displayed)
- Assign Labels to Points
- Export